# ILP Experimentation Example Notebook
This notebook provides examples of loading, processing and charting the result json file from an experiment.

### Basic Imports
The following imports are necessary for loading and processing the data.

In [ ]:
import json
from ilpexp.result import decode_result

### Charting / Numeric Analysis Imports
These imports are necessary for charting and also some of the numerical analysis.
To run them you should be sure you have `matplotlib` and `numpy` installed. This can be done on most machines using `pip`
```bash
pip3 install matplotlib
pip3 install numpy
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean, median

plt.rcParams['figure.figsize'] = (12,6)

## Loading A Results File
In the following cell, change the file location in `RESULTS_FILE` as appropriate and run the cell. This will result in two variables: 

* `results` is the original list of `ExperimentResult` objects parsed from JSON.
* `stitched_results` is a nested dictionary indexed by `problem` then `system`. For each problem and system there is a list of `ExperimentResult` objects corresponding to the trials for that system and problem.

In [ ]:
RESULTS_FILE = './results/results.json'

"""
    The JSON results are a simple list of instance run results. This function turns them into
    a nested dictionary of lists of the form {problem}->{system}->[trials].

    This structure can make it easier to analyze individual problems.
"""
def stitch_results(results):
    stitched_results = {}
    for result in results:
        stitched_result = stitched_results.setdefault(result.problem_name, {})
        stitched_result_list = stitched_result.setdefault(result.system_name, [])
        stitched_result_list.append(result)
    
    for problem in stitched_results.values():
        for system in problem.values():
            system.sort(key = lambda x: x.trial if x.trial else 0)
    
    return stitched_results

"""
    Loads the results from the given filename.
"""
def load_results_from_json(filename):
    with open(filename) as file:
        json_string = file.read()
    return json.loads(json_string, object_hook=decode_result)

"""
    Loads and stitches the results from the given filename
"""
def load_results(filename):
    results = load_results_from_json(filename)
    return (results, stitch_results(results))

(results, stitched_results) = load_results(RESULTS_FILE)

## Table Generation
This cell processes the data into summary data suitable for a table. The data is stored in a variable called `table_results`.

In [ ]:
TIMES = "times"
ACCURACY = "accuracy"

def accuracy(result):
    if result.conf_matrix is not None:
        (tp, fn, tn, fp) = result.conf_matrix
        return ((tp + tn)* 100)/(tp + fn + tn + fp)
    return 50

def std_err(lst):
    data = np.array(lst)
    return np.std(data, ddof=1) / np.sqrt(np.size(data))

def std_dev(lst):
    data = np.array(lst)
    return np.std(data, ddof=1)

def get_table_data(stitched_results):
    table_results = {}
    for (problem_name, system_dict) in stitched_results.items():
        table_system_dict = table_results.setdefault(problem_name, {})
        for (system_name, trials) in system_dict.items():
            system_summary = table_system_dict.setdefault(system_name, {})

            times = [x.total_exec_time for x in trials]
            accuracies = [accuracy(x) for x in trials]

            system_summary[TIMES] = (mean(times), std_err(times))
            system_summary[ACCURACY] = (mean(accuracies), std_dev(accuracies))
            
    return table_results

table_results = get_table_data(stitched_results)

## Latex Table Output
This cell outputs the table results in Latex. The names associated with each problem and system identifier can be adjusted by editing `SYSTEM_NAMES` and `PROBLEM_NAMES` respectively before running the cell.


In [ ]:
SYSTEM_NAMES = {
    "popper": "Popper",
    "aleph": "Aleph",
    "metagol": "Metagol"
}

"""
    The default behaviour doesn't change the problem names. Mappings can be added here if needed.
"""
PROBLEM_NAMES = {
    "example": "Example"
}

def latex_format_times(d):
    return f"{d[TIMES][0]:.1f} $\\pm$ {d[TIMES][1]:.1f}"

def latex_format_accuracies(d):
    return f"{d[ACCURACY][0]:.0f} $\\pm$ {d[ACCURACY][1]:.0f}"

def make_line(name, lst):
    return f"{name} & " + " & ".join(lst)

def output_latex_table(table_results, format_func):
    # This assumes the systems are the same for all problems. The table wouldn't work well otherwise.
    systems = list(table_results.values())[0].keys()
    system_names = [SYSTEM_NAMES.get(name, name) for name in systems]
    columns = "l |" + "|".join([" c "]*len(system_names))
    table_begin = "\\begin{table}[ht]\n\\footnotesize\n\\centering\n\\begin{tabular}{" + columns + "}\n\\toprule\n"
    table_end = "\\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}"
    header_names = ["\\textbf{" + name+ "}" for name in system_names]
    lines = [
        ("Task", header_names),
    ]
    for (problem_name, problem_systems) in table_results.items():
        vals = []
        lines.append((PROBLEM_NAMES.get(problem_name, problem_name), vals))
        
        for system in systems:
            system_summary = problem_systems[system]
            vals.append(format_func(system_summary))

    table = table_begin + "\\\\\n".join([make_line(start, lst) for (start, lst) in lines]) + table_end
    print(table)

print("MEAN EXECUTION TIMES\n----------------\n")
output_latex_table(table_results, format_func=latex_format_times)

print("\n\nMEAN ACCURACIES\n----------------\n")
output_latex_table(table_results, format_func=latex_format_accuracies)

## Basic Charting
The following cells show a few examples of using matplotlib to draw some simple charts of the results. These aren't exactly useful for arbitrary systems and problems but might be helpful examples when using the experimental results to dig a bit deeper into some systems or problems.

In [ ]:
def grouped_bars(labels, vals1, vals2, ylabel, title, bar_label_fmt):
    x = np.arange(len(labels))  
    width = 0.35

    _, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, vals1[0], width, label=vals1[1], hatch=".", edgecolor='black')
    rects2 = ax.bar(x + width/2, vals2[0], width, label=vals2[1], hatch="x",edgecolor='black')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend(handlelength=4, fontsize='large')

    #ax.bar_label(rects1, padding=3, fmt=bar_label_fmt)
    #ax.bar_label(rects2, padding=3, fmt=bar_label_fmt)

    plt.show()

def line_chart(labels, data, xlabel, ylabel, title, legend_loc='lower right', markevery=1):
    for vals, label, marker in data:
        plt.plot(labels, vals, label=label, marker=marker, markevery=markevery)
    if len(data) > 1:
        plt.legend(loc=legend_loc)
    if len(data) > 1:
        plt.legend(loc=legend_loc)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    plt.show()

def scatter(x, y, xlabel, ylabel, title):
    plt.scatter(x, y)
    xlim = plt.xlim()[1]
    plt.xlim(0, xlim)
    plt.ylim(0, xlim)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel(xlabel) 
    plt.ylabel(ylabel)
    plt.title(title) 

    plt.show()

In [ ]:
def draw_execution_time_bar_chart(table_results):
    # Create a fixed ordering of problems since they are stored in dicts and need to be transformed to lists
    problems = table_results.keys()
    labels = [PROBLEM_NAMES.get(name, name) for name in problems]

    # The chart only works with two systems so this example just arbitrarily takes the first two to extablish an order.
    systems = list(list(table_results.values())[0].keys())[0:2]
    system_names = [SYSTEM_NAMES.get(name, name) for name in systems]

    vals = []
    for system in systems:
        times = []
        vals.append(times)
        for problem in problems:
            system_summary = table_results[problem][system]
            # Append mean execution time
            times.append(system_summary[TIMES][0])

    grouped_bars(labels, 
            (vals[0], system_names[0]),
            (vals[1], system_names[1]),
            'Execution Time',
            f'Execution Time for {system_names[0]} and {system_names[1]}',
            '%.1f')

draw_execution_time_bar_chart(table_results)

In [ ]:
def draw_execution_time_scatter_plot(stitched_results):
    problem, problem_systems = list(stitched_results.items())[0]
    problem_name = PROBLEM_NAMES.get(problem, problem)

    systems = list(problem_systems.items())[0:2]
    system_names = [SYSTEM_NAMES.get(name, name) for (name, _) in systems]

    vals = []
    for (_, system) in systems:
        times = [trial.total_exec_time for trial in system]
        vals.append(times)

    scatter(vals[0], vals[1], system_names[0], system_names[1], f"Execution Times For Trials of {problem_name}")

draw_execution_time_scatter_plot(stitched_results)

## Popper Extra Stats
The `extra_stats` field on an `ExperimentResult` contains extra details about the execution of an instance. 

Currently this field is only populated by the Popper system.

Below are some example functions that show how the extra stats can be used for the current version of Popper.

In [ ]:
"""
    Returns a dict containing details of the last stage of execution for Popper. A stage is a search for a satisfying program with a certain number of literals.
    In this case, the last stage represents the search in which Popper either found a solution or identified that no solutions exist.
"""
def last_stage(result):
    stages = result.extra_stats["stages"]
    sorted_stages = sorted(stages, key=lambda x: x["num_literals"], reverse=True)
    return sorted_stages[0]

"""
    This function returns the number of literals in the solution (or the last number of literals Popper tried if there was no solution.)
"""
def num_literals(result):
    return last_stage(result)["num_literals"]

"""
    This function returns the total number of programs Popper tested before finding a solution or failing.
"""
def total_programs(result):
    return last_stage(result)["total_programs"]

"""
    This function returns the total duration of a given phase of the Popper loop.
"""
def duration(result, phase):
    return sum(result.extra_stats["durations"][phase])

In [ ]:
# This is an example of how the previous cell's functions can be used. It relies on there being a dropk problem for a system called "popper" with at least one trial.
# This may not be the case in all experiments, so it is meant for illustrative purposes.
popper_dropk_first_trial = stitched_results["dropk"]["popper"][0]

print(total_programs(popper_dropk_first_trial))
print(num_literals(popper_dropk_first_trial))
print(duration(popper_dropk_first_trial, "generate"))